#### From scratch simple Rnn layer equivaleent to pytorch's Rnn layer


In [1]:

from torch.nn import Module
from torch import nn
import torch.nn.functional as F
import math
import torch
from torch import nn

class MyRNN(nn.Module):
    def __init__(self, input_size, hidden_size):
        super().__init__()
        self.Wih = torch.nn.Parameter(torch.empty(hidden_size, input_size))
        self.Whh = torch.nn.Parameter(torch.empty(hidden_size, hidden_size))
        self.outs = []
        self.bias_ih = nn.Parameter(torch.zeros(input_size))
        self.bias_hh = nn.Parameter(torch.zeros(hidden_size))
        
        nn.init.kaiming_uniform_(self.Wih, a=math.sqrt(5))
        nn.init.kaiming_uniform_(self.Whh, a=math.sqrt(5))

        self.activation = nn.Tanh()

    def forward(self, sequence):
        self.outs = []
        sequences_num = sequence.size(0)

        for i in range(sequences_num):
           hx = F.linear(sequence[i], self.Wih, self.bias_ih)
           hh = F.linear(self.outs[-1] if i > 0 else torch.zeros(hx.size()), self.Whh, self.bias_hh)
           h = self.activation(hx + hh)
           self.outs.append(h) 
        outs = torch.stack(self.outs)
        return outs, self.outs[-1]

In [2]:
features_size = 16
hidden_size = 7
rnn = MyRNN(input_size=features_size, hidden_size=hidden_size)

torch_rnn = nn.RNN(input_size=features_size, hidden_size=hidden_size)

rnn.Wih = torch_rnn.weight_ih_l0
rnn.Whh = torch_rnn.weight_hh_l0
rnn.bias_ih = torch_rnn.bias_ih_l0
rnn.bias_hh = torch_rnn.bias_hh_l0

In [3]:
sequence = torch.randn(3, features_size)
a_out, a_last = rnn(sequence)
b_out, b_last = torch_rnn(sequence)

torch.allclose(a_out, b_out), torch.allclose(a_last, b_last)

(True, True)